<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;"><b>Treball de Final de Màster </b></p>
<p style="margin: 0; text-align:right;"><b>ANÀLISI DEL DIA MUNDIAL DE LES MALALTIES MINORITÀRIES (Font de dades: TWITTER)</b>.</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PAC 3: Disseny i Implementació del TFM.

# PREPROCESSAT i 'FEATURE ENGINEERING' DEL DATASET DE DADES: 
<p style="color:#0000FF">
En aquest Jupyter Notebook, es recullen les proves amb TF-IDF. Millora de
    Bag of Words. Revisió del recompte i càlcul del factor IDF.
</p>

<div style="text-align:right">
    <span style="font-family:calibri;font-size:8;color:white; background-color:Crimson; text-align:right; margin-left: 0.5cm;padding:0.3cm"> Autor: </span>
    <span style="font-family:calibri;font-size:8;color:white; background-color:RoyalBlue; text-align:center;  margin-right: 0.5cm; padding:0.3cm"><b> Joaquim de Dalmases Juanet </b></span>
</div>

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize, WordPunctTokenizer, RegexpTokenizer, TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.lancaster import LancasterStemmer
from datetime import datetime
from datetime import timedelta  
import string
import re, collections
import time
from textblob import TextBlob
import codecs
import json
import emoji
import gensim
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
from IPython.display import Image as im

In [4]:
# Càrrega del dataset generat en la fase de preprocessat.
time_start = time.time()
dataset=pd.read_excel("DMMM_Dataset_final.xlsx")
temps=(time.time()-time_start)/60
print("#Total de tuits:",dataset.shape[0],"\n Durada: ",int(temps) if temps>0 else 0,"minut/s ", \
      int((temps-int(temps))*60),"segons.")

#Total de tuits: 98433 
 Durada:  0 minut/s  24 segons.


In [3]:
dataset.head(2)

,Unnamed: 0,_id,created_at,text_x,text_net,text_Norm,diaSem,dia,mes,yy,...,user_idstr,user_friends_c,user_followers_c,user_listed_c,retweet_count,lang,polarity,subjectivity,emojis,text_y
0,0,5e78e80f4e54db2148d19749,Thu Feb 13 09:58:19 +0000 2020,RT @FEDER_ONG: Las 29 obras ganadoras del conc...,feder_ong winning works photographic contest...,feder_ong win work photograph contest eresar...,Thursday,13,2,2020,...,4054806561,8,3,0,0,es,0.500000,0.750000,[],RT @FEDER_ONG: The 29 winning works of the pho...
1,1,5e78e80f4e54db2148d1974a,Thu Feb 13 10:01:01 +0000 2020,"📌 Esta tarde, a partir de las 21:00h, nuevo #C...",afternoon starting h new cronichat centered ...,afternoon start h new cronich cent enfermeda...,Thursday,13,2,2020,...,64270216,853,2104,31,0,es,0.068182,0.277273,['📌'],"📌 This afternoon, starting at 21 :00h, new #Cr..."


In [4]:
def fa_wordcloud(ftext,bkcolor='white',ample=15,alt=10):    
    wordcloud = WordCloud(background_color=bkcolor).generate(ftext)
    plt.figure(figsize=(ample,alt))
    plt.imshow(wordcloud,interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [6]:
#nombre de tuits
tuits=list(dataset.text_net)
len(tuits)

98433

In [7]:
tuits[0]

'  feder_ong winning works photographic contest eresarte form travelling exhibition tour geography espa'

In [8]:
cv = CountVectorizer()
wc=cv.fit_transform(tuits)

In [9]:
wc.shape

(98433, 39251)

In [17]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)

In [18]:
tfidf_transformer.fit(wc)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [19]:
# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

,idf_weights
rarediseaseday,1.923264
rare,2.152886
day,2.514935
today,2.700182
disease,2.889772
...,...
gemmageo,11.803994
gemmachand,11.803994
profesorjmb,11.803994
proudrarediseasemommoment,11.803994


In [20]:
tf_idf_vector=tfidf_transformer.transform(wc)

In [24]:
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df1=df.sort_values(by=["tfidf"],ascending=False)

In [27]:
df1[df1.tfidf>0]

,tfidf
geography,0.374200
espa,0.374200
travelling,0.355353
winning,0.316786
tour,0.291945
photographic,0.287174
contest,0.271266
eresarte,0.254702
form,0.242617
works,0.234266


In [28]:
first_document_vector.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.]])